In [1]:
from utils import *

In [2]:
with open('pickles/go_dict.pkl', 'r') as fin:
    go_dict = pickle.load(fin)
with open('pickles/protein_dict.pkl', 'r') as fin:
    protein_dict = pickle.load(fin)
with open('pickles/gene_dict.pkl', 'r') as fin:
    gene_dict = pickle.load(fin)

In [3]:
import cPickle as pickle
import pandas as pd

In [4]:
columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['allergen'] == True and v['sequence'] not in allergen_set:
        allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

columns = protein_dict.itervalues().next().keys()
columns_data = [[] for c in columns]
non_allergen_set = set()
for k,v in protein_dict.iteritems():
    if v['sequence'] not in allergen_set and v['sequence'] not in non_allergen_set:
        non_allergen_set.add(v['sequence'])
        for i,c in enumerate(columns):
            columns_data[i].append(v[c])
uniprot_non_allergen_df = pd.DataFrame(dict(tuple(zip(columns, columns_data))))

In [5]:
uniprot_allergen_df.head(1)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,Q4WCS7,True,Swiss-Prot,1.11.1.15,PMP20_ASPFU,"[pmp20, AFUA_6G02280]",330879,[O43099],Putative peroxiredoxin pmp20,MSGLKAGDSFPSDVVFSYIPWSEDKGEITACGIPINYNASKEWADK...


In [6]:
uniprot_non_allergen_df.head(1)

,acc_num,allergen,db,ec_num,entry_name,gene_names,organism_name,other_acc_nums,protein_name,sequence
0,C6DJ78,False,Swiss-Prot,None,TTCA_PECCP,"[ttcA, PC1_2316]",561230,[],tRNA 2-thiocytidine biosynthesis protein TtcA,MSENQQINQKQQYNLNKLQKRLRRNVGEAIADFNMIEEGDRIMVCL...


In [7]:
print uniprot_allergen_df.shape, uniprot_non_allergen_df.shape

(601, 10) (464738, 10)


In [8]:
fasta_columns = ['db', 'identifier', 'entry_name', 'protein_name', 'organism_name', 'gene_name', 'sequence']
parsed_fasta = parse_fasta_str(open('data/allergen_online_allergens.fasta').read())
ao_df = pd.DataFrame(data=parsed_fasta, columns=fasta_columns)

ao_identifiers = ao_df['identifier'].tolist()
for i in range(0, len(ao_identifiers), 200):
    if i == 0:
        ao_df = convert_acc_nums_to_df(str(ao_identifiers[i:i+200]))
    else:
        ao_df = ao_df.append(convert_acc_nums_to_df(str(ao_identifiers[i:i+200])))

In [9]:
ao_df[ao_df['db'] == 'tr'].head(1)

,db,identifier,entry_name,protein_name,organism_name,gene_name,sequence
0,tr,B0KZJ6,B0KZJ6_ACASI,Allergen Aca s 13,Acarus siro,None,MVQINGSYKLEKSDNFDAFLKELGLNFVTRNLAKSATPTVEVSVNG...


In [10]:
#includes entries from the tremble database as well as swiss-prot
ao_set = set(ao_df['sequence'])
print len(ao_set)
print len(ao_set.difference(allergen_set))
print len(ao_set.difference(non_allergen_set))
print len(ao_set.difference(allergen_set, non_allergen_set))

582
405
529
352


In [11]:
final_allergen_sequences = [s for s in allergen_set.union(ao_set.difference(non_allergen_set))] 

In [12]:
allergen_df = pd.DataFrame({'sequence': final_allergen_sequences})
allergen_df.shape

(953, 1)

In [13]:
non_allergen_sequences = [s for s in non_allergen_set.difference(ao_set)]
non_allergen_df = pd.DataFrame({'sequence': non_allergen_sequences})
non_allergen_df.shape

(464685, 1)

In [14]:
with open('pickles/allergen_df.pkl', 'w') as fout:
    pickle.dump(allergen_df, fout)
    
with open('pickles/non_allergen_df.pkl', 'w') as fout:
    pickle.dump(non_allergen_df, fout)

In [15]:
# Uncomment this code to run this cell without running whole notebook
# with open('pickles/non_allergen_df.pkl', 'r') as fin:
#     non_allergen_df = pickle.load(fin)
# with open('pickles/allergen_df.pkl', 'r') as fin:
#     allergen_df = pickle.load(fin)

allergen_df['allergen'] = 1
non_allergen_df['allergen'] = 0
allergen_df = add_protein_characteristics(allergen_df)
non_allergen_df = add_protein_characteristics(non_allergen_df)

with open('pickles/allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(allergen_df, fout)
    
with open('pickles/non_allergen_df_with_features.pkl', 'w') as fout:
    pickle.dump(non_allergen_df, fout)